In [42]:
import warnings
warnings.filterwarnings("ignore")
import gmsh
import meshio
gmsh.initialize()

In [43]:
gmsh.model.add("3D")
L, B, H = 10, 10, 10
r = 2
channel = gmsh.model.occ.addBox(0, 0, 0, L, B, H)

In [44]:
sphere = gmsh.model.occ.addSphere(5, 5, 5, r)

In [45]:
fluid = gmsh.model.occ.cut([(3, channel)], [(3, sphere)])

In [46]:
gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(dim=3)

In [47]:
marker = 11
gmsh.model.addPhysicalGroup(volumes[0][0], [volumes[0][1]], marker)
gmsh.model.setPhysicalName(volumes[0][0], marker, "conductor")

In [48]:
gmsh.model.occ.synchronize()
gmsh.model.mesh.refine()
gmsh.model.mesh.generate(3)

In [49]:
gmsh.write("mesh3D.msh")

In [50]:
def create_mesh(mesh, cell_type, prune_z=False):
    cells = mesh.get_cells_type(cell_type)
    cell_data = mesh.get_cell_data("gmsh:physical", cell_type)
    points = mesh.points[:,:2] if prune_z else mesh.points
    out_mesh = meshio.Mesh(points=points, cells={cell_type: cells}, cell_data={"name_to_read":[cell_data]})
    return out_mesh

In [51]:
mesh_3d = meshio.read("mesh3D.msh")
tetrahedral_mesh = create_mesh(mesh_3d, "tetra")
meshio.write("mesh/10_0_10/mesh_tetr.xdmf", tetrahedral_mesh)

In [ ]:
cmd_3d = 'mpirun -n 2 python3 ion_transport.py --working_dir=/home/lesh/dev/ssb/ --grid_info=10_0_10 --file_shape=10_10'
!{cmd_3d}